# Create dataset for the model, using binary matrix representation

In [1]:
import numpy as np
import pandas as pd 
import pickle
%matplotlib inline
import matplotlib.pyplot as plot
import matplotlib.image as mpimg
import time

from numba import cuda,vectorize,float64,jit

## Read the preprocessed dataFrame

In [2]:
#Read pikle into DF
file_name = 'final_DF_for_stim_classification'
df = pd.read_pickle('../data_tidying_analysis/' + file_name)

In [23]:
df[]

,stimType,stimId,sampleId,X_axis,Y_axis
0,Snacks,1,bmem_short_101_1_1,93,330
1,Snacks,1,bmem_short_101_1_1,101,262
2,Snacks,1,bmem_short_101_1_1,123,182
3,Snacks,1,bmem_short_101_1_1,142,147
4,Snacks,1,bmem_short_101_1_1,148,128
5,Snacks,1,bmem_short_101_1_1,149,124
6,Snacks,1,bmem_short_101_1_1,158,125
7,Snacks,1,bmem_short_101_1_1,161,130
8,Snacks,1,bmem_short_101_1_1,157,127
9,Snacks,1,bmem_short_101_1_1,156,125


## Creat the binary matrix for each sampleId and feet it to DF along with the label

In [3]:
df_no_duplicates = df.drop_duplicates()

In [6]:
sampleIds = df_no_duplicates.sampleId.unique()
labelsData = [None] * len(sampleIds)
samplesData = [None] * len(sampleIds)
i = 0
for sampleId in sampleIds:
    matrix = np.zeros((401, 401))
    indexes = df[df['sampleId'] == sampleId].index
    print('Log...... buildeing matrix for sample ID: ' + sampleId)
    for index in indexes:
        matrix[df[df.index == index].X_axis, df[df.index == index].Y_axis] = 1.0  
    labelsData[i] = sampleId.split('_')[-1]
    samplesData[i] = matrix
    print('Log...... added the binary matrix to the sample and the label to the list')
    i = i + 1
    #plot.imshow(matrix)
    #plot.show()

Log...... buildeing matrix for sample ID: bmem_short_101_1_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_101_2_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_101_3_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_101_4_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_101_5_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_101_6_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_101_7_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_10

Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_101_68_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_101_70_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_101_72_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_101_73_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_101_74_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_101_75_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_101_76_1
Log...... added the binary matrix to the sample 

Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_105_31_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_105_32_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_105_33_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_105_35_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_105_36_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_105_40_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_105_41_1
Log...... added the binary matrix to the sample 

Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_106_33_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_106_34_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_106_35_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_106_36_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_106_38_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_106_39_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_106_40_1
Log...... added the binary matrix to the sample 

Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_107_21_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_107_22_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_107_23_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_107_24_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_107_25_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_107_26_1
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_107_27_1
Log...... added the binary matrix to the sample 

...

Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_141_99_3
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_141_100_2
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_141_101_2
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_141_103_3
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_141_104_2
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_141_105_3
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_141_106_3
Log...... added the binary matrix to the s

Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_142_5_2
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_142_6_3
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_142_8_2
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_142_10_3
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_142_13_2
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_142_15_2
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_142_16_3
Log...... added the binary matrix to the sample and

Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_142_102_3
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_142_103_2
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_142_104_3
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_142_107_2
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_142_109_3
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_142_111_2
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_142_113_2
Log...... added the binary matrix to the 

Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_143_9_2
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_143_10_3
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_143_12_2
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_143_13_2
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_143_14_2
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_143_16_2
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_143_17_3
Log...... added the binary matrix to the sample a

Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_143_129_2
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_143_130_3
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_143_132_3
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_143_133_2
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_143_134_2
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_143_138_2
Log...... added the binary matrix to the sample and the label to the list
Log...... buildeing matrix for sample ID: bmem_short_143_139_3
Log...... added the binary matrix to the 

## Save labels list and samples list to pkl file

In [11]:
with open('labels_data.pickle', 'wb') as handle:
    pickle.dump(labelsData, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [15]:
with open('samples_data.pickle', 'wb') as handle:
    pickle.dump(samplesData, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
#check smaplesData size
import sys
sys.getsizeof(samplesData)

44032

In [16]:
# Because the datset exceeds limit (2GB) we use this custome method to split and append the bytes to 'filepath'.
def save_as_pickled_object(obj, filepath):
    """
    This is a defensive way to write pickle.write, allowing for very large files on all platforms
    """
    max_bytes = 2**31 - 1
    bytes_out = pickle.dumps(obj)
    n_bytes = sys.getsizeof(bytes_out)
    with open(filepath, 'wb') as f_out:
        for idx in range(0, n_bytes, max_bytes):
            f_out.write(bytes_out[idx:idx+max_bytes])

In [17]:
save_as_pickled_object(samplesData, 'sample_data_cmpressed')